# Simulation of Urban Agriculture



In the following exercise, we will create a simulation of an efficient trade model for self-produced goods. The model will consider consumer preferences and calculate the social utility of these goods based on all individual preferences. This involves calculating the price of the social utility for each good and the total production. A monetary base is then computed, and a percentage of it is distributed to individuals using the same parameters, taking into account their individual production. Finally, the model simulates various production scenarios with the aim of minimizing the total remaining balance for all individuals.

## Individuals

Each individual is has two inmutable variables in the model: **preferences matrix** and **production space**. Although this two can change in real life for the moment they will stay inmutable. 


### Preferences Matrix

Given $k$ preferences over $n$ goods, we will construct a preference matrix with dimensions $n \times k$. To simplify calculations, we will work with a square matrix by taking $n$ preferences. Each column of the preference matrix represents a good, and each row represents a normalized basket of preferences. We assume that all these preference baskets provide the same utility to the individual and are calculated on a 100-point preference system (assigning a value out of 100 points to each good). It's important to note that this preference matrix does not assume rationality among preferences and offers a different approach to understanding market dynamics.

### Production Space

Additionally,the production space represents the amount of space each individual can allocate for production. We will assume that each individual's production space is fully utilized for each term.


On the other hand, all individuals have three time-dependent variables: a money balance and a production vector, and a space occupation vector.

### Money Balance

The money balance begans in $0$ at $t = 0$ and accumulates over time.

### Production Vector

The production vector is a variable that indicates how the space is occupied and how much time is remaining for the production of each good.

## Goods, Money and Prices

